This notebook is using data prep/encoding that we decided upon for group project.  We have divided algorithm analysis up between members and want to have uniform data prep/encoding for comparison.

In [36]:
# Important Statements
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [37]:
dataset = pd.read_csv("bank-additional-full.csv")
#inital EDA showed strong correlation for emp.var.rate/nr.employed with euribor3m, duration can't know ahead of time, also drop
dataset = dataset.drop(['emp.var.rate','nr.employed', 'duration'], axis=1)

In [38]:
dataset.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'campaign', 'pdays', 'previous',
       'poutcome', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'y'],
      dtype='object')

In [39]:
dataset.shape

(41188, 18)

In [40]:
dataset.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,cons.price.idx,cons.conf.idx,euribor3m,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,93.994,-36.4,4.857,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,93.994,-36.4,4.857,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,93.994,-36.4,4.857,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,93.994,-36.4,4.857,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,93.994,-36.4,4.857,no


In [41]:
#Initial data prep
#check for null values
#dataset.isnull().values.any()
#remove rows with >= 4 unkown values
#dataset = dataset.replace(to_replace='unknown', value=np.nan).dropna(thresh=17)
#dataset = dataset.replace(to_replace='unknown', value=np.nan).dropna()
#dataset = dataset.replace(to_replace=np.nan, value='unknown')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 18 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
y                 41188 non-null object
dtypes: float64(3), int64(4), object(11)
memory usage: 5.7+ MB


In [42]:
#One Hot Encoding all Catergorical Variables without Order
import category_encoders as ce

encoder = ce.BinaryEncoder(cols=['job','marital','education','default','housing','loan','contact','poutcome','month','day_of_week'])
#encoder = ce.BinaryEncoder(cols=['job','marital','default','housing','loan','contact','poutcome','month','day_of_week'])
dataset = encoder.fit_transform(dataset)

In [43]:
# Encoding Outcome Row to Binary
#dataset['outcome'] = dataset['outcome'].map({'yes': 1, 'no': 0})
dataset['y'] = dataset['y'].map({'yes': 1, 'no': 0})

In [44]:
#dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 43 columns):
age               41188 non-null int64
job_0             41188 non-null int64
job_1             41188 non-null int64
job_2             41188 non-null int64
job_3             41188 non-null int64
job_4             41188 non-null int64
marital_0         41188 non-null int64
marital_1         41188 non-null int64
marital_2         41188 non-null int64
education_0       41188 non-null int64
education_1       41188 non-null int64
education_2       41188 non-null int64
education_3       41188 non-null int64
default_0         41188 non-null int64
default_1         41188 non-null int64
default_2         41188 non-null int64
housing_0         41188 non-null int64
housing_1         41188 non-null int64
housing_2         41188 non-null int64
loan_0            41188 non-null int64
loan_1            41188 non-null int64
loan_2            41188 non-null int64
contact_0         41188 non-null 

In [45]:
dataset.shape

(41188, 43)

In [46]:
# Defining Dependent and Independent values
X = dataset.iloc[:,:-1].values 
y = dataset.iloc[:,-1].values 

In [47]:
#Splitting the data into Training Set and Test Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [48]:
# Scaling
from sklearn.preprocessing import StandardScaler 
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [49]:
#Fitting Classifier to Training Set. Create a classifier object here and call it classifierObj
from sklearn.ensemble import RandomForestClassifier

#classifierObj= RandomForestClassifier()
#grid_param= {  
#    'n_estimators': [10, 15, 20, 25, 30, 40, 50],
#    'criterion': ['gini', 'entropy'],
#    'bootstrap': [True, False]
#}
#from sklearn.model_selection import GridSearchCV
#gd_sr= GridSearchCV(estimator=classifierObj, param_grid=grid_param, scoring='accuracy', cv=5, n_jobs=-1)
#gd_sr.fit(X_train, y_train) 
#print(gd_sr.best_params_) 
#print(gd_sr.best_score_)

#grid search results {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 50}
#0.9110773899848255

classifierObj= RandomForestClassifier(bootstrap=True, criterion='gini', n_estimators=50)
classifierObj.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [50]:
#K-Fold Cross Validation
from sklearn.model_selection import cross_val_score
modelAccuracies = cross_val_score(estimator=classifierObj, X=X_train, y=y_train, cv=10)
print(modelAccuracies.mean())
print(modelAccuracies.std())

0.8932018097874346
0.003230609291601723


In [51]:
#Making predictions on the Test Set
y_pred = classifierObj.predict(X_test)

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[7094  225]
 [ 660  259]]


In [52]:
score = classifierObj.score(X_test, y_test)
print(score)

0.892571012381646
